# <font color="Blue">Autotrader Machine Learning Predictive Modelling</font>

This Notebook implements a standard Machine Learning Pipeline to address a real-world Machine Learning problem: **predicting the selling price of a car**
using the "Car Sale Adverts" Dataset provided by Autotrader. 

The steps to achieve this are broken down below: 

1. **Data/Domain Understanding and Exploration:** Loading, analysing and visualising the datasets features. 

2. **Data Processing for Machine Learning:** Cleaning the data (handling missing data, outliers and noise) and performing feature engineering and transformations. 

3. **Model Building:** Fitting, tuning and selectig models. This will include k-Nearest Neighbours, Decision Tree Classifier, Linear Regression and Random Forest. 

4. **Model Evaluation and Analysis:** Evaluating the various models using variouis metrics, analysing feature importance, and inspecting individual predictions erros.


***

## <font color="Blue">Packages</font>

The following section of code imports all relevant Python packages grouped by functionality. 
    

In [ ]:
# Import Data Processing Libraries
import pandas as pd
import numpy as np

# Import Visualisation Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Import Machine Learning Libraries
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

# Other Imports 
from datetime import datetime

## <font color="Blue">Initial Data Exploration</font>

The following cells perform an initial exploration of the data. 

In [ ]:
# Load the dataset
df = pd.read_csv('adverts.csv')

In [ ]:
# Output the first 5 observations
df.head()

In [ ]:
# Sample Data
df.sample(5)

In [ ]:
# Print out dataframe information 
df.info()

In [ ]:
# Data Inspection and Classification 
# Function to Classify Features 
def summarise_features(df, max_cardinality=100):
    # Empty list to store summary data for each column
    rows = []
    # Loops through each column name in the DataFrame
    for col in df.columns:
        # Select entire column as a series
        s = df[col]
        # Get datatype of column
        dtype = str(s.dtype)
        # Get number of unique values
        n_unique = s.nunique(dropna=True)
        # Calculate percentage of mising values by using mean on null values rounded to 3 decimal places
        pct_missing = round(s.isna().mean(), 3)
        # Check if data type is numerical
        if pd.api.types.is_numeric_dtype(s):
            # Set features type
            ftype = 'numerical'
            # Check if data is a datetime object
        elif pd.api.types.is_datetime64_any_dtype(s):
            # Set feature type to datetime
            ftype = 'datetime'
        else: 
            # Check if feature is categorical (can use one hot encoding) or high_cardinality (can't use onehot encoding)
            ftype = 'categorical' if n_unique <= max_cardinality else 'high_cardinality'
            # Append dictionary of column data to rows list
        rows.append({
            'column': col,
            'dtype': dtype,
            'feature_type': ftype,
            'n_unique': n_unique,
            'pct_missing': pct_missing
        })
        # Convert rows list to a DataFrame and return
    return pd.DataFrame(rows).sort_values(['feature_type', 'n_unique'], ascending=[True, False])

feature_summary = summarise_features(df)
display(feature_summary)

In [ ]:
# Exploratory Data Visualisation and Analysis

# Feature Target
target = 'price'
# Drop ID Features
cols_to_drop = ['public_reference', 'reg_code']

# Create list of numerical column names 
num_cols = [c for c in df.select_dtypes(include='number').columns if c not in cols_to_drop]
# Create a list of categorical feature names
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# 1. Histograms 
# Loop through numerical columns
for col in num_cols:
    # Create blank 6x3 figure
    plt.figure(figsize=(6, 3))
    # Create histogram for column, dropping null values 
    sns.histplot(df[col].dropna(), kde=True, bins=30)
    plt.title(f'{col} distribution')
    plt.show() 

# 2. Boxplots to inspect outliers (numerical)
# Create blank 12x4 figure
plt.figure(figsize=(12, 4))
# Draw  boxplots using a random sample (up to 1000 rows) for speed and orient them horizontally.
sns.boxplot(data=df[num_cols].sample(n=min(1000, len(df))), orient='h')
plt.title('Boxplots (sampled)')
plt.show()

# 3. Correlation heatmap (numerical)
# Create balck 10x8 figure
plt.figure(figsize=(10, 8))
# Calculate correlation matrix for all numerical columns
corr = df[num_cols].corr()
# Use seaborn to draw heatmap of calculated correlation matrix, 
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', square=True, vmin=-1, vmax=1)
plt.title('Numeric feature correlations')
plt.show()

# 4. Scatter plots vs target for all numerical features
# Check if the target variable exists and is in the list of numerical columns
if target in df.columns and target in num_cols:
    # Create a list of the numerical feature columns by excluding the target 
    features_to_plot = [col for col in num_cols if col != target]

    # Loop through each numerical feature
    for c in features_to_plot:
        # Create a 6x4 figure for each plot
        plt.figure(figsize=(6,4))
        # Draw a scatter plot with the feature and the target
        sns.scatterplot(x=df[c], y=df[target], alpha=0.5)
        plt.title(f'{c} vs {target}')
        plt.show()

# 5. Countplots for categorical features (top categories)
# Loop through first 6 categorical features
for c in cat_cols[:6]:  
    # Top = the 10 most common cetegories in the column 
    top = df[c].value_counts().nlargest(10).index
    # Create 8x4 figure
    plt.figure(figsize=(8,4))
    # Draw bar chart of top most categorical features
    sns.countplot(y=c, order=top, data=df)
    plt.title(f'Top categories for {c}')
    plt.show()

# 6. Missingness Map
# This provides a visual map of the entire raw dataset to show where missing values are.
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Initial Missing Value Matrix of Raw Data')
plt.show()

# 7. Boxplots of target vs categorical features (top categories)
# Loop through each categorical column
for c in cat_cols:
    # To keep plots readable plot only the Top 10 most frequent categories 
    # 1. Get the names of the top 10 categories by frequency
    top_10_cats = df[c].value_counts().nlargest(10).index    
    # 2. Filter the DataFrame to only include rows with these top categories
    # Use .copy() to avoid a SettingWithCopyWarning
    df_top_10 = df[df[c].isin(top_10_cats)].copy()

    # 3. Create the boxplot
    plt.figure(figsize=(10, 5))
    sns.boxplot(x=c, y=target, data=df_top_10, order=top_10_cats)
    
    plt.title(f'Price Distribution by {c} (Top 10 Categories)')
    plt.xticks(rotation=45) 
    plt.show()


## <font color="Blue">Data Processing for Better Visualisation</font>

Data processing for visualisation involves transforming and refining raw data to enhance the clarity and interpretability of exploratory plots. Unlike preprocessing for machine learning models, the primary goal here is to reveal underlying patterns and trends that might otherwise be obscured by noise, extreme outliers, or highly skewed scales.

To facilitate this analysis, the following specific processing steps were implemented:

**Feature Engineering & Cleaning:** A new vehicle_age feature was derived from the registration year to provide a more intuitive metric for depreciation. Additionally, invalid data (such as registration years prior to the invention of cars in 1886) was filtered out.

**Normalising Skewed Distributions:** Logarithmic transformations were applied to the price and mileage features to correct their heavy right-skew, allowing for a clearer view of the central data distribution.

**Outlier Management:** Extreme outliers in numerical columns were clipped to the 1st and 99th percentiles to prevent anomalies from distorting the scale of the graphs.

**Binning Continuous Data:** The continuous mileage feature was grouped into categorical bins (e.g. '0-5k', '5k-20k') to facilitate the analysis of price trends across different life cycle stages of the vehicles.

In [ ]:
# Data Processing and Feature Engineering for Visualisation

# Create a copy of the dataframe for visualisation purposes to keep the original df clean.
df_vis = df.copy()

# Remove obvious Errors 
# Cars were invented in 1886, therefore, it is logical to assume any vehicle older 1886 is an Error
# Also, any future year is invalid
df_vis.loc[(df_vis['year_of_registration'] < 1886) | (df_vis['year_of_registration'] > datetime.now().year), 'year_of_registration'] = np.nan

# Create vehicle_age from the registration year (current_year - year_of_registration) tunrning anything that can't be parsed to NaN instead of throwing an error
df_vis['vehicle_age'] = datetime.now().year - pd.to_numeric(df_vis['year_of_registration'], errors='coerce')

# Log transform right skewed numeric columns to improve distribution plots
for c in ['price', 'mileage']:
    if c in df_vis.columns and df_vis[c].min(skipna=True) >= 0:
        df_vis[f'log_{c}'] = np.log1p(df_vis[c])

# Clip extreme outliers in numeric columns for display (Top and Bottom 1%) 
for c in df_vis.select_dtypes(include='number').columns:
    # Compute 1st and 99th percentiles
    lo = df_vis[c].quantile(0.01)
    hi = df_vis[c].quantile(0.99)
    # Replace values outside the bounds with the boundary values
    df_vis[c] = df_vis[c].clip(lower=lo, upper=hi)

# Create mileage bins for easier plotting 
df_vis['mileage_bin'] = pd.cut(df_vis['mileage'], bins=[-1, 5000, 20000, 50000, 100000, 300000], labels=['0-5k','5k-20k','20k-50k','50k-100k','100k+'])

### <font color="Blue">Data Visualisation after Processing for Better Visualisation </font>

In [ ]:
# Data Visualisation after Preprocessing

# Plot vehicle age
plt.figure(figsize=(6, 3))
if 'vehicle_age' in df_vis.columns:
    sns.histplot(df_vis['vehicle_age'].dropna(), bins=30)
    plt.title('vehicle age')
    plt.xlabel('vehicle age')
    plt.show()
else:
    print("Column 'vehicle_age' not found in df_vis. Run the preprocessing cell first.")

# Log transformed numerical columns
log_cols_to_plot = [c for c in ['log_price', 'log_mileage'] if c in df_vis.columns]

# Histograms of Transformed Numerical Columns
for col in log_cols_to_plot:
    plt.figure(figsize=(6, 3))
    sns.histplot(df_vis[col].dropna(), kde=True, bins=30)
    plt.title(f'{col} distribution')
    plt.xlabel(col)
    plt.show()

# Top 10 categories Boxplots
for c in cat_cols:
    # Ensure the column exists in df_viz
    if c not in df_vis.columns:
        continue
    top_10_cats = df_vis[c].value_counts().nlargest(10).index
    if len(top_10_cats) == 0:
        continue
    df_top_10 = df_vis[df_vis[c].isin(top_10_cats)].copy()
    plt.figure(figsize=(10, 5))
    if 'log_price' in df_vis.columns:
        sns.boxplot(x=c, y='log_price', data=df_top_10, order=top_10_cats)
        plt.title(f'Price Distribution by {c} (Top 10 Categories)')
        plt.xticks(rotation=45)
        plt.show()
    else:
        print("Column 'log_price' not found in df_vis; skipping boxplot for {c}.")

# Plot price by mileage_bin using a boxplot
plt.figure(figsize=(10,4))
if 'mileage_bin' in df_vis.columns and 'log_price' in df_vis.columns:
    sns.boxplot(x='mileage_bin', y='log_price', data=df_vis)
    plt.xticks(rotation=45)
    plt.title('Price by mileage bin')
    plt.show()
else:
    print("Columns 'mileage_bin' or 'log_price' missing in df_vis; ensure preprocessing ran.")

## <font color="Blue">Data Processing for Machine Learning</font>

This phase focused on transforming the raw dataset into a clean, numerical format suitable for model training while strictly adhering to best practices for preventing data leakage. Unlike exploratory analysis where the whole dataset may be viewed, processing for machine learning requires that all distributional parameters (such as medians for imputation, quantiles for outlier clipping, and means for scaling) be derived solely from the training set.

To achieve this, the data processing was structured into two distinct stages:

Stateless Transformations: Row-wise operations that do not depend on other samples, such as deriving vehicle_age from registration data and removing obvious errors (e.g. years < 1886), were performed globally to ensure data consistency.

Stateful Transformations via Pipelines: The dataset was split into Training (80%) and Testing (20%) sets. Scikit-learn Pipelines were then constructed to encapsulate distinct processing logical flows for numerical and categorical data. This ensured that steps such as Median Imputation, Outlier Clipping (using a custom transformer based on the 1st and 99th percentiles of the training set), and Standard Scaling were fit only on X_train and legally applied to X_test.

In [ ]:
# Data Processing for Machine Learning (operate on a copy `df_ml` to preserve `df`)

# Define a function to derive the registration year from the 'reg_code'
def reg_code_to_year(reg_code):
    try:
        code = int(float(reg_code))
        if 51 <= code <= 99:
            return 2000 + (code - 50)
        elif 0 < code <= 50:
            return 2000 + code
        else:
            return np.nan
    except (ValueError, TypeError):
        return np.nan

# Work on a copy to avoid mutating the original `df`
df_ml = df.copy()

# Impute year or registration with derived year using reg_code_to_year function 
df_ml['derived_year'] = df_ml['reg_code'].apply(reg_code_to_year)
df_ml['year_of_registration'] = df_ml['year_of_registration'].fillna(df_ml['derived_year']) if 'year_of_registration' in df_ml.columns else df_ml.get('year_of_registration')

# Create vehicle age from year of registration since it is a more intuitive feature
df_ml['vehicle_age'] = datetime.now().year - pd.to_numeric(df_ml.get('year_of_registration', pd.Series()), errors='coerce')

# Create log price again
df_ml['log_price'] = np.log1p(df_ml['price'])

# Drop unneeded columns and clean
cols_to_drop = ['public_reference', 'reg_code', 'year_of_registration', 'derived_year']
df_ml.drop(columns=[c for c in cols_to_drop if c in df_ml.columns], inplace=True, errors='ignore')


# Drop columns missing import features for prediciting
correlated_missing_cols = [c for c in ['standard_colour', 'vehicle_condition', 'vehicle_age'] if c in df_ml.columns]
if correlated_missing_cols:
    df_ml.dropna(subset=correlated_missing_cols, how='all', inplace=True)

# Drop duplicate columns
df_ml.drop_duplicates(inplace=True)

# Create missingness indicators on df_ml
for c in df_ml.columns:
    if df_ml[c].isna().any():
        df_ml[f'{c}_missing'] = df_ml[c].isna().astype(int)

# Define features (X) by dropping the target and related columns from df_ml
cols_to_exclude = ['price', 'log_price', 'mileage_bin']
X = df_ml.drop(columns=cols_to_exclude, axis=1, errors='ignore')
y = df_ml['log_price'] 

# Ensure X and y are aligned by dropping rows where the target value is missing
if hasattr(y, 'isnull') and y.isnull().any():
    valid_indices = y.dropna().index
    X = X.loc[valid_indices]
    y = y.loc[valid_indices]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

# Copies to avoid SettingWithCopyWarning
X_train = X_train.copy()
X_test = X_test.copy()

# Numerical and categorical columns from training set
num_cols_to_impute = X_train.select_dtypes(include=['number']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Imputation
# Learn medians from the training set to avoid data leakage
# Impute missing numerical data using the median
imputer_num = SimpleImputer(strategy='median')
# Fit the imputere on the training data and transforn it 
X_train[num_cols_to_impute] = imputer_num.fit_transform(X_train[num_cols_to_impute])
# Fit the imputer on the test data using the median learned from the test data
X_test[num_cols_to_impute] = imputer_num.transform(X_test[num_cols_to_impute])

# Impute the missing categorical data with the mode
imputer_cat = SimpleImputer(strategy='most_frequent')
# Fit the imputer on the training data and transform it 
X_train[cat_cols] = imputer_cat.fit_transform(X_train[cat_cols])
# Transform the test data using the mode learned from the training data
X_test[cat_cols] = imputer_cat.transform(X_test[cat_cols])

# Outlier clipping from training set to prevent data leakage
num_cols_for_clipping = X_train.select_dtypes(include=['number']).columns
for c in num_cols_for_clipping:
    lo = X_train[c].quantile(0.01)
    hi = X_train[c].quantile(0.99)
    X_train[c] = X_train[c].clip(lower=lo, upper=hi)
    X_test[c] = X_test[c].clip(lower=lo, upper=hi)

# Group rare categories into 'Other' based on training frequencies to prevent data leakage
for c in cat_cols:
    freqs = X_train[c].value_counts(normalize=True)
    rares = freqs[freqs < 0.01].index
    X_train[c] = X_train[c].replace(list(rares), 'Other')
    X_test[c] = X_test[c].replace(list(rares), 'Other')

# Frequency enoding 
for c in cat_cols:
    freq_map = X_train[c].value_counts(normalize=True)
    X_train[f'{c}_freq'] = X_train[c].map(freq_map)
    X_test[f'{c}_freq'] = X_test[c].map(freq_map).fillna(0)

# Identify and Handle Low vs High Cardinality Categorical Features 
# High cardinality features >100 unique values will use target encoding (mean encoding) to avoid creating sparse high-dimensional outputs
# Low cardinality features <=100 unique values will use one-hot encoding

max_cardinality_threshold = 100
low_card_cols = []
high_card_cols = []

# Seperate categorical columns based on cardinality
for c in cat_cols:
    n_unique = X_train[c].nunique()
    if n_unique <= max_cardinality_threshold:
        low_card_cols.append(c)
    else:
        high_card_cols.append(c)

# Apply target encoding (mean encoding) to high cardinality features
target_encodings = {}
for c in high_card_cols:
    target_means = y_train.groupby(X_train[c]).mean()
    target_encodings[c] = target_means
    # Map categories to their mean target value; fill unmapped values with overall mean
    X_train[c] = X_train[c].map(target_means).fillna(y_train.mean())
    X_test[c] = X_test[c].map(target_means).fillna(y_train.mean())

# Update categorical columns list to only include low cardinality features for one hot encoding
cat_cols = low_card_cols
# Add high cardinality features to numerical columns, they are numeric after target encoding
num_cols_to_impute.extend(high_card_cols)

# Preprocessor for OneHotEncoding and Standard Scaler
# This step prepares a 'preprocessor' object that will be used in the model building phase
# It encapsulates the final transformations, scaling and one hot encoding

# Get the final lists of numerical and categorical columns after the previous steps
num_colsF = X_train.select_dtypes(include=['number']).columns.tolist()
cat_colsF = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Define the transformer for one hot encoding categorical features (low-cardinality only)
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# Define the transformer for standardising numerical features
numeric_transformer = StandardScaler()

# Combine the transformers into a single ColumnTransformer object
# This object is not fitted here. It will be fitted on the training data within the model pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_colsF),
        ('cat', categorical_transformer, cat_colsF)
    ],
    verbose_feature_names_out=False
)


### <font color="Blue">Data Visualisation after Data Processing for Machine Learning</font>

This section serves as a critical quality check, visualising the training data after all machine learning preprocessing steps have been applied. The aim is to verify that the transformations were effective before feeding the data to the models. The following plots confirm the success of key operations: the missing value heatmap validates that imputation is complete, histograms show the improved distributions of numerical features, and countplots display how rare categories were successfully grouped into an 'Other' category. These checks ensure the data is clean, robust, and properly formatted for modeling.

In [ ]:
# Data Visualisation after Preprocessing for Machine Learning

# The following plots use the X_train and y_train data to reflect the changes

# Combine X_train and y_train for plotting purposes
train_df_for_plotting = X_train.copy()
train_df_for_plotting['log_price'] = y_train

# Missingness Map after imputation
plt.figure(figsize=(12, 6))
sns.heatmap(train_df_for_plotting.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Value Matrix for Training Data after Imputation')
plt.show()

# Histograms of Imputed Numerical Columns
# These plots show the distribution of key numerical features after missing values were filled
print("Distribution of Numerical Features in the Training Set after imputation")
imputed_num_cols_to_plot = ['mileage', 'vehicle_age']
for col in imputed_num_cols_to_plot:
    plt.figure(figsize=(8, 4))
    sns.histplot(train_df_for_plotting[col], kde=True, bins=30)
    plt.title(f'Distribution of {col} in Training Set')
    plt.show()

# Countplots for Categorical Features after grouping rare categories
# These plots show the new 'Other' category which groups all rare categories together
print("\nDistribution of Categorical Features in the Training Set after grouping")
for c in cat_cols: 
    plt.figure(figsize=(10, 5))
    # Order the bars by frequency
    order = train_df_for_plotting[c].value_counts().index
    sns.countplot(y=c, data=train_df_for_plotting, order=order)
    plt.title(f'Top Categories for {c} in Training Set after grouping rares')
    plt.show()

# Boxplots of Target vs. Processed Categorical Features
# This shows how the price is distributed across the processed categorical features, including the 'Other' group
print("\nPrice Distribution by Processed Categorical Features")
for c in cat_cols: 
    plt.figure(figsize=(12, 6))
    # To keep plots readable use the value_counts index which is already ordered by frequency
    order = train_df_for_plotting[c].value_counts().index
    sns.boxplot(x=c, y='log_price', data=train_df_for_plotting, order=order)
    plt.title(f'Price Distribution by {c} in Training Set')
    plt.xticks(rotation=45)
    plt.show()

# Visualising Categorical Features without the 'Other' Category 
# Since the 'Other' category is very large and dominates the plots, make plots without it to better see the data
for c in cat_cols:
    # Create a temporary DataFrame for plotting that filters out the 'Other' category
    df_filtered = train_df_for_plotting[train_df_for_plotting[c] != 'Other']
    
    if not df_filtered.empty:
        # Countplot without 'Other' 
        plt.figure(figsize=(10, 5))
        order = df_filtered[c].value_counts().index
        sns.countplot(y=c, data=df_filtered, order=order)
        plt.title(f"Top Categories for {c} without other")
        plt.show()

        # Boxplot without 'Other'
        plt.figure(figsize=(12, 6))
        order = df_filtered[c].value_counts().index
        sns.boxplot(x=c, y='log_price', data=df_filtered, order=order)
        plt.title(f"Price Distribution by {c} without other")
        plt.xticks(rotation=45)
        plt.show()
    else:
        print(f"Skipping plots for '{c}' as it only contains the 'Other' category after filtering.")
        
# Histogram of a Target Encoded Column
# This shows the distribution of the mean target values that replaced the original categories
if high_card_cols: # Check if there are any high cardinality columns
    col_to_plot = high_card_cols[0] # Plot the first one
    plt.figure(figsize=(8, 4))
    sns.histplot(train_df_for_plotting[col_to_plot], kde=True, bins=30)
    plt.title(f'Distribution of Target-Encoded Feature: {col_to_plot}')
    plt.xlabel('Mean Log Price')
    plt.show()

## <font color="Blue">Model Building</font>

This section transitions from data preparation to the core task of predictive modeling. Here, three different regression models; K-Nearest Neighbors, Decision Tree, and Linear Regression and an ensemble; Random Forestm, are constructed and trained to predict vehicle prices. To ensure robustness and prevent data leakage, each model is embedded within a Scikit-learn Pipeline that chains the previously defined data preprocessor with the regressor. For the K-Nearest Neighbors and Decision Tree models, `GridSearchCV` is employed to systematically explore a range of hyperparameters and identify the optimal configuration for each, using cross-validation on a subset of the training data for efficiency. The Linear Regression model is evaluated using cross-validation before being trained on the entire training dataset. The final, best-performing version of each model is stored for the subsequent evaluation phase.

In [ ]:
# Model Building 

# Instantiate Models as Pipelines 
# Create a pipeline for K-Nearest Neighbors
knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor())
])

# Create a pipeline for Decision Tree
dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=99))
])

# Create a pipeline for Linear Regression
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Create a pipeline for Random Forest Regressor
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
]) 

# Hyperparameter grids for GridSearch 
# Define the hyperparameter grid for KNN
# Search for the best number of neighbors and weight scheme (uniform or distance which weights closer neighbours higher)
knn_param_grid = {
    'regressor__n_neighbors': [3, 5, 7, 9, 11],
    'regressor__weights': ['uniform', 'distance']
}

# Define the hyperparameter grid for Decision Tree
dt_param_grid = {
    'regressor__max_depth': [5, 10, 15, None],
    'regressor__min_samples_leaf': [1, 2, 4],
}

# Define the hyperparameter grid for Random Forest
rf_param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [10, 20, None],
    'regressor__min_samples_split': [2, 5],
}


# Train Models and Tune using GridSearchCV
print("Starting Model Training and Hyperparameter Tuning")

# Dictionary to store the fitted models
fitted_models = {}
# Keep grid search objects for later inspection
grid_searches = {}

# Grid Search Function
def grid_search(pipeline, 
                param_grid, 
                X, y, 
                cv=3, 
                scoring={'r2': 'r2', 'neg_root_mean_squared_error': 'neg_root_mean_squared_error'}, 
                refit='r2', 
                name='model'):
    gs = GridSearchCV(pipeline, 
                      param_grid, 
                      cv=cv, 
                      scoring=scoring, 
                      refit=refit, 
                      n_jobs=-1, 
                      return_train_score=True, 
                      verbose=1)
    gs.fit(X, y)
    return gs

# To accelerate hyperparameter tuning train GridSearchCV on a smaller, random sample of the training data
# This provides a good estimate of the best parameters much more quickly
search_frac = 0.5
X_search = X_train.sample(frac=search_frac, random_state=99)
y_search = y_train.loc[X_search.index]
print(f"Note: Using a random {int(search_frac*100)}% subset of the training data ({len(X_search)} samples) to speed up tuning.")

# K-Nearest Neighbors Tuning
print("\nTuning K-Nearest Neighbors...")
knn_grid_search = grid_search(knn_pipeline, knn_param_grid, X_search, y_search, cv=3, name='KNN')

# Store the best model found
fitted_models['K-Nearest Neighbors'] = knn_grid_search.best_estimator_
grid_searches['K-Nearest Neighbors'] = knn_grid_search

# Output results
print("\nK-Nearest Neighbors Tuning Complete.")
print(f"Best Parameters Found for KNN: {knn_grid_search.best_params_}")
print(f"Best Cross-Validation R-squared Score: {knn_grid_search.best_score_:.4f}")
best_rmse_knn = -knn_grid_search.cv_results_['mean_test_neg_root_mean_squared_error'][knn_grid_search.best_index_]
print(f"Best Cross-Validation RMSE (log scale): {best_rmse_knn:.4f}")


# Print top 5 configurations
print("\nTop 5 KNN Configurations:")
knn_results_df = pd.DataFrame(knn_grid_search.cv_results_).sort_values(by='rank_test_r2')
knn_results_df['mean_test_rmse'] = -knn_results_df['mean_test_neg_root_mean_squared_error']
knn_results_df['std_test_rmse'] = knn_results_df['std_test_neg_root_mean_squared_error']
display(knn_results_df[['params', 'mean_test_r2', 'std_test_r2', 'mean_test_rmse', 'std_test_rmse']].head())


# Decision Tree Tuning using sampled data to speed up tuning
print('\nTuning Decision Tree...')
dt_grid_search = grid_search(dt_pipeline, dt_param_grid, X_search, y_search, cv=3, name='Decision Tree')

# Store the best model found
fitted_models['Decision Tree'] = dt_grid_search.best_estimator_
grid_searches['Decision Tree'] = dt_grid_search

# Output results
print("\nDecision Tree Tuning Complete.")
print(f'Best Parameters Found for Decision Tree: {dt_grid_search.best_params_}')
print(f'Best Cross-Validation R-squared Score: {dt_grid_search.best_score_:.4f}')
best_rmse_dt = -dt_grid_search.cv_results_['mean_test_neg_root_mean_squared_error'][dt_grid_search.best_index_]
print(f"Best Cross-Validation RMSE (log scale): {best_rmse_dt:.4f}")

# Display top 5 configurations
print("\nTop 5 Decision Tree Configurations:")
dt_results_df = pd.DataFrame(dt_grid_search.cv_results_).sort_values(by='rank_test_r2')
dt_results_df['mean_test_rmse'] = -dt_results_df['mean_test_neg_root_mean_squared_error']
dt_results_df['std_test_rmse'] = dt_results_df['std_test_neg_root_mean_squared_error']
display(dt_results_df[['params', 'mean_test_r2', 'std_test_r2', 'mean_test_rmse', 'std_test_rmse']].head())


# Linear Regression Evaluation 
print('\nEvaluating Linear Regression...')
# Linear Regression doesn't require extensive tuning, but cross validation is still helpful
lr_cv_scores_r2 = cross_val_score(lr_pipeline, X_train, y_train, cv=3, scoring='r2', n_jobs=-1)
print(f'Linear Regression Cross-Validation R-squared scores: {lr_cv_scores_r2}')
print(f'Linear Regression Mean CV R-squared: {lr_cv_scores_r2.mean():.4f} (+/- {lr_cv_scores_r2.std():.4f})')

# Calculate RMSE using cross-validation
lr_cv_scores_rmse = cross_val_score(lr_pipeline, X_train, y_train, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
print(f'\nLinear Regression Cross-Validation RMSE scores (log scale): {-lr_cv_scores_rmse}')
print(f'Linear Regression Mean CV RMSE (log scale): {-lr_cv_scores_rmse.mean():.4f} (+/- {lr_cv_scores_rmse.std():.4f})')

# Fit the final linear regression model on the full training set
print("\nFitting final Linear Regression model on the full training data...")
lr_pipeline.fit(X_train, y_train)
fitted_models['Linear Regression'] = lr_pipeline
print("Linear Regression model fitted.")

# Random Forest Tuning using sampled data to speed up tuning
print('\nTuning Random Forest...')
rf_grid_search = grid_search(rf_pipeline, rf_param_grid, X_search, y_search, cv=3, name='Random Forest')

# Store the best model found
fitted_models['Random Forest'] = rf_grid_search.best_estimator_
grid_searches['Random Forest'] = rf_grid_search

# Output results
print("\nRandom Forest Tuning Complete.")
print(f'Best Parameters Found for Random Forest: {rf_grid_search.best_params_}')
print(f'Best Cross-Validation R-squared Score: {rf_grid_search.best_score_:.4f}')
best_rmse_rf = -rf_grid_search.cv_results_['mean_test_neg_root_mean_squared_error'][rf_grid_search.best_index_]
print(f"Best Cross-Validation RMSE (log scale): {best_rmse_rf:.4f}")

# Display top 5 configurations
print("\nTop 5 Random Forest Configurations:")
rf_results_df = pd.DataFrame(rf_grid_search.cv_results_).sort_values(by='rank_test_r2')
rf_results_df['mean_test_rmse'] = -rf_results_df['mean_test_neg_root_mean_squared_error']
rf_results_df['std_test_rmse'] = rf_results_df['std_test_neg_root_mean_squared_error']
display(rf_results_df[['params', 'mean_test_r2', 'std_test_r2', 'mean_test_rmse', 'std_test_rmse']].head())

# Final Summary 
print('\nModel building and tuning complete.')
print('The best versions of all models are now stored in the fitted_models dictionary for analysis.')

## <font color="Blue">Model Evaluation and Analysis</font>

This final stage provides a comprehensive evaluation of the trained models on the unseen test data. The analysis begins by calculating key performance metrics specifically Test Set R-squared (R²) and Root Mean Squared Error (RMSE) to create a quantitative comparison and identify the best performing model.

To gain deeper insights into what drives the models' predictions, a Feature Importance Analysis is conducted. For the Linear Regression model, the feature coefficients are extracted and visualised to show the magnitude and direction of their influence. For the Decision Tree, the feature_importances_ attribute is used to rank features by their contribution to the model's predictive power.

Finally, an Error Analysis is performed using residual plots, which graph the prediction errors against the predicted values. This helps diagnose any systematic issues and provides a better understanding of each model's strengths and weaknesses beyond single performance metrics.

In [ ]:
# Model Evaluation and Analysis

# Make Predictions and Evaluate Performance 
# List to store performance metrics for each model
results = []
# Dictionary used to store the actual predicted values for error analysis and residual plots
predictions = {}

# Iterate thorugh models to; make predictions, calculate metrics and print summary 
for name, model in fitted_models.items():
    print(f"Evaluating {name}...")
    
    # Make predictions on the test data.
    y_pred = model.predict(X_test)
    predictions[name] = y_pred
    
    # Inverse transform log_price values to original price scale
    y_test_actual = np.expm1(y_test)
    y_pred_actual = np.expm1(y_pred)
    
    # Calculate evaluation metrics
    mse = mean_squared_error(y_test_actual, y_pred_actual)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_actual, y_pred_actual)
    mape = mean_absolute_percentage_error(y_test_actual, y_pred_actual)
    r2 = r2_score(y_test, y_pred)
    
    # Append metrics to the results list
    results.append({
        'Model': name,
        'Test Set R-squared (R2)': r2,
        'Test Set RMSE (£)': rmse,
        'Test Set MAE (£)': mae,
        'Test Set MAPE (%)': mape * 100
    })
    
    # Print metrics
    print(f"  R-squared: {r2:.4f}")
    print(f"  Root Mean Squared Error: £{rmse:,.2f}")
    print(f"  Mean Absolute Error: £{mae:,.2f}")
    print(f"  Mean Absolute Percentage Error: {mape*100:.2f}%\n")

# Compare Models
# Convert the results to a DataFrame for easy comparison
results_df = pd.DataFrame(results).sort_values(by='Test Set R-squared (R2)', ascending=False)

print("Final Model Comparison on Test Set")
display(results_df)

# Visualise Predictions vs Actual Values for All Models (Scatter Plots)
print("\nActual vs Predicted Values for All Models (Scatter Plot)")

for name, y_pred in predictions.items():
    # Inverse transform to actual price scale
    y_test_actual = np.expm1(y_test)
    y_pred_actual = np.expm1(y_pred)
    
    # Scatter plot of predicted vs actual
    plt.figure(figsize=(8, 8))
    sns.scatterplot(x=y_test_actual, y=y_pred_actual, alpha=0.5)
    plt.plot([y_test_actual.min(), y_test_actual.max()], 
             [y_test_actual.min(), y_test_actual.max()], 
             '--', color='blue', linewidth=2, label='Perfect Prediction')
    plt.title(f'"{name}" - Actual vs Predicted Prices (£)')
    plt.xlabel('Actual Price (£)')
    plt.ylabel('Predicted Price (£)')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Visualise Best Model's Predictions vs Actual Values (in £ scale)
best_model_name = results_df.iloc[0]['Model']
best_model = fitted_models[best_model_name]
y_pred_best = best_model.predict(X_test)

# Inverse transform BOTH to actual £ scale
y_test_actual_best = np.expm1(y_test)
y_pred_best_actual = np.expm1(y_pred_best)

# DEBUG: Check the ranges
print(f"Actual prices - Min: £{y_test_actual_best.min():,.0f}, Max: £{y_test_actual_best.max():,.0f}, Mean: £{y_test_actual_best.mean():,.0f}")
print(f"Predicted prices - Min: £{y_pred_best_actual.min():,.0f}, Max: £{y_pred_best_actual.max():,.0f}, Mean: £{y_pred_best_actual.mean():,.0f}")
print(f"Y_test (log) - Min: {y_test.min():.2f}, Max: {y_test.max():.2f}")
print(f"Y_pred (log) - Min: {y_pred_best.min():.2f}, Max: {y_pred_best.max():.2f}")

# Scatter plot
plt.figure(figsize=(10, 8))
sns.scatterplot(x=y_test_actual_best, y=y_pred_best_actual, alpha=0.5)
plt.plot([y_test_actual_best.min(), y_test_actual_best.max()], 
         [y_test_actual_best.min(), y_test_actual_best.max()], 
         '--', color='red', linewidth=2, label='Perfect Prediction')
plt.title(f'"{best_model_name}" - Actual vs Predicted Prices (£)')
plt.xlabel('Actual Price (£)')
plt.ylabel('Predicted Price (£)')
plt.legend()
plt.tight_layout()
plt.show()

# Feature Importance Analysis 
print("\nFeature Importance Analysis")

# Linear Regression, plot coeeficients to show top 15 features with the biggest effect on the price
lr_model = fitted_models.get('Linear Regression')
# Get feature names and coefficients
feature_names = lr_model.named_steps['preprocessor'].get_feature_names_out()
coefficients = lr_model.named_steps['regressor'].coef_

# Create dataframe to pair each feature with its coefficient 
coef_df = pd.DataFrame({'feature': feature_names, 'coefficient': coefficients})
# Calculate the absolute value of the coefficients 
coef_df['abs_coefficient'] = np.abs(coef_df['coefficient'])
# Sort in descending order and select top 15
coef_df = coef_df.sort_values('abs_coefficient', ascending=False).head(15)
        
# Barplots of the top 15 features
plt.figure(figsize=(10, 8))
sns.barplot(x='coefficient', y='feature', data=coef_df)
plt.title('Top 15 Feature Importances for Linear Regression')
plt.xlabel('Coefficient')
plt.ylabel('Feature')
plt.show()

# Decision Tree, plot features that best reduced impurity in leaf nodes, best predictors
dt_model = fitted_models.get('Decision Tree')
# Get feature names and importances
feature_names = dt_model.named_steps['preprocessor'].get_feature_names_out()
importances = dt_model.named_steps['regressor'].feature_importances_
        
# Dataframe to pair each feature with its importance 
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False).head(15)

# Plot barplots of best predictors  
plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=importance_df)
plt.title('Top 15 Feature Importances for Decision Tree')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

# Random Forest feature importance
rf_model = fitted_models.get('Random Forest')
if rf_model:
    feature_names = rf_model.named_steps['preprocessor'].get_feature_names_out()
    importances = rf_model.named_steps['regressor'].feature_importances_
    
    rf_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
    rf_importance_df = rf_importance_df.sort_values('importance', ascending=False).head(15)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(x='importance', y='feature', data=rf_importance_df)
    plt.title('Top 15 Feature Importances for Random Forest')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.show()

# Error Analysis, Residual Plots
print("\nError Analysis: Residual Plots")

# Make plots of actual - predicted
for name, y_pred in predictions.items():
    residuals = y_test - y_pred
    
    # Plot scatter plot of predicted values and residuals with line of perfect prediction
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=y_pred, y=residuals, alpha=0.5)
    plt.axhline(y=0, color='red', linestyle='--')
    plt.title(f'Residual Plot for {name}')
    plt.xlabel('Predicted Values (log_price)')
    plt.ylabel('Residuals (Actual - Predicted)')
    plt.show()

In [ ]:
# Filtered Actual vs Predicted Plots (Outliers Removed)
print("FILTERED PLOTS: Actual vs Predicted Values (Outliers Removed)")
print("These plots exclude data points beyond the 1st and 99th percentiles to highlight typical model performance.\n")

for name, y_pred in predictions.items():
    # Inverse transform to actual price scale
    y_test_actual = np.expm1(y_test)
    y_pred_actual = np.expm1(y_pred)
    
    # Calculate percentiles for filtering (1st and 99th) using numpy percentile
    actual_p1, actual_p99 = np.percentile(y_test_actual, [1, 99])
    pred_p1, pred_p99 = np.percentile(y_pred_actual, [1, 99])
    
    # Create boolean mask for points within both ranges
    mask = (y_test_actual >= actual_p1) & (y_test_actual <= actual_p99) & \
           (y_pred_actual >= pred_p1) & (y_pred_actual <= pred_p99)
    
    # Filter data
    y_test_filtered = y_test_actual[mask]
    y_pred_filtered = y_pred_actual[mask]
    
    # Count outliers removed
    n_removed = (~mask).sum()
    pct_removed = (n_removed / len(mask)) * 100
    
    # Create filtered scatter plot
    plt.figure(figsize=(8, 8))
    sns.scatterplot(x=y_test_filtered, y=y_pred_filtered, alpha=0.5)
    plt.plot([y_test_filtered.min(), y_test_filtered.max()], 
             [y_test_filtered.min(), y_test_filtered.max()], 
             '--', color='red', linewidth=2, label='Perfect Prediction')
    plt.title(f'"{name}" - Actual vs Predicted Prices (£)\n(Filtered: {n_removed} outliers removed, {pct_removed:.1f}%)')
    plt.xlabel('Actual Price (£)')
    plt.ylabel('Predicted Price (£)')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Print statistics for filtered data
    print(f"{name} (Filtered):")
    print(f"  Points removed: {n_removed} ({pct_removed:.1f}%)")
    print(f"  Actual prices - Min: £{y_test_filtered.min():,.0f}, Max: £{y_test_filtered.max():,.0f}")
    print(f"  Predicted prices - Min: £{y_pred_filtered.min():,.0f}, Max: £{y_pred_filtered.max():,.0f}\n")

# Filtered plot for Best Model
print(f"FILTERED PLOT: Best Model ({best_model_name})")

y_test_actual_best = np.expm1(y_test)
y_pred_best_actual = np.expm1(y_pred_best)

# Calculate percentiles using numpy percentile
actual_p1, actual_p99 = np.percentile(y_test_actual_best, [1, 99])
pred_p1, pred_p99 = np.percentile(y_pred_best_actual, [1, 99])

# Create mask
mask_best = (y_test_actual_best >= actual_p1) & (y_test_actual_best <= actual_p99) & \
            (y_pred_best_actual >= pred_p1) & (y_pred_best_actual <= pred_p99)

# Filter data
y_test_best_filtered = y_test_actual_best[mask_best]
y_pred_best_filtered = y_pred_best_actual[mask_best]

n_removed_best = (~mask_best).sum()
pct_removed_best = (n_removed_best / len(mask_best)) * 100

# Create plot
plt.figure(figsize=(10, 8))
sns.scatterplot(x=y_test_best_filtered, y=y_pred_best_filtered, alpha=0.5)
plt.plot([y_test_best_filtered.min(), y_test_best_filtered.max()], 
         [y_test_best_filtered.min(), y_test_best_filtered.max()], 
         '--', color='red', linewidth=2, label='Perfect Prediction')
plt.title(f'"{best_model_name}" - Actual vs Predicted Prices (£) - Filtered\n({n_removed_best} outliers removed, {pct_removed_best:.1f}%)')
plt.xlabel('Actual Price (£)')
plt.ylabel('Predicted Price (£)')
plt.legend()
plt.tight_layout()
plt.show()